In [1]:
import os, sys, time, random
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions
from utils.wrapped_xgboost import xgboost_classifier
from utils.validation_tools import score_MCC, MCC, create_validation_index
from utils.models import CombinedModel
from utils.data_munge import remove_single_value_columns
from utils.feature_engineering import NumericalFeatureEngineering, getRelativeTimeColumns, BasicDate_FeatureEngineering
from utils.feature_engineering import getTimeChangeColumns, getTimeSteps, build_IndexFeatures, build_sortedData_indexDiff

data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'

start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

In [2]:
tot_row_num = 1183747
num_rows = 5000
skip = sorted(random.sample(xrange(1,tot_row_num + 1),tot_row_num - num_rows))

In [3]:
#'''
start_time = time.time()
## loading the data by using the skipped_row_num list
#train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', nrows=num_rows)
#train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', nrows=num_rows)
#train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', nrows=num_rows)

## randomly select certain rows
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', skiprows=skip, nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', skiprows=skip, nrows=num_rows)
train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', skiprows=skip, nrows=num_rows)

test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id', nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id', nrows=num_rows)
test_cat = pd.read_csv(join(data_path, test_cat_file),      index_col='Id', nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
#'''

finish loading date using 88.0 seconds


In [5]:
remove_single_value_columns(train_num, 'Response', test=test_num)
remove_single_value_columns(train_dat, test=test_dat)
remove_single_value_columns(train_cat, test=test_cat)

raw train data dimension:  (5000, 969)
raw test data dimension:  (5000, 968)
processed train data dimension:  (5000, 969)
processed test data dimension:  (5000, 968)
raw train data dimension:  (5000, 1156)
raw test data dimension:  (5000, 1156)
processed train data dimension:  (5000, 1145)
processed test data dimension:  (5000, 1145)
raw train data dimension:  (5000, 2140)
raw test data dimension:  (5000, 2140)
processed train data dimension:  (5000, 750)
processed test data dimension:  (5000, 750)


In [6]:
dat_columns = train_dat.columns.tolist()
num_columns = train_num.columns.tolist()
num_columns.remove(dep_var_name)

def build_column_dict(columns):
    station_dict = {}
    line_dict = {}
    for col in columns:
        stationList = col.split('_')[0:2]
        stationKey = ('_').join(stationList)
        lineKey = col.split('_')[0]
        
        if lineKey not in line_dict:
            line_dict[lineKey] = [col]
        else:
            line_dict[lineKey].append(col)
                    
        if stationKey not in station_dict:
            station_dict[stationKey] = [col]
        else:
            station_dict[stationKey].append(col)
            
    return station_dict, line_dict


def build_station_features(df, col_dict, prefix='dat'):
    features = pd.DataFrame()
    for key, value in col_dict.items():
        features['{}_{}_{}'.format(prefix, key, 'mean')] = df[value].mean(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'max')] = df[value].max(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'min')] = df[value].min(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'var')] = df[value].var(axis=1)
    return features


def build_station_index_features(train, test = None):
    selected_columns = []
    for col in train.columns:
        if 'mean' in col or 'var' in col:
            selected_columns.append(col)
            
    if test is not None:
        train_test = pd.concat([train[selected_columns], test[selected_columns]], axis=0)
    else:
        train_test = train[selected_columns]
        
    train_test['index'] = train_test.index
    #new_fea = pd.DataFrame()
    ## function to build index based on the given columns
    new_fea = build_sortedData_indexDiff(train_test, selected_columns, index_col_name = 'index')
    
    return new_fea


dat_col_dict, dat_line_dict = build_column_dict(dat_columns)
num_col_dict, num_line_dict = build_column_dict(num_columns)

dat_col_dict.update(dat_line_dict)
num_col_dict.update(num_line_dict)

In [7]:
start_time = time.time()

train_dat_stations = build_station_features(train_dat, dat_col_dict, 'dat')
test_dat_stations = build_station_features(test_dat, dat_col_dict, 'dat')

train_num_stations = build_station_features(train_num, num_col_dict, 'num')
test_num_stations = build_station_features(test_num, num_col_dict, 'num')

print 'finish feature engineering date station using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date station using 0.05 minutes


In [8]:
num_station_index = build_station_index_features(train_num_stations, test_num_stations)
dat_station_index = build_station_index_features(train_dat_stations, test_dat_stations)

In [9]:
combined_train_station_num = pd.concat([train_num_stations, num_station_index.ix[train_num_stations.index]], axis=1)
combined_train_station_dat = pd.concat([train_dat_stations, dat_station_index.ix[train_dat_stations.index]], axis=1)

combined_test_station_num = pd.concat([test_num_stations, num_station_index.ix[train_num_stations.index]], axis=1)
combined_test_station_dat = pd.concat([test_dat_stations, dat_station_index.ix[train_dat_stations.index]], axis=1)

In [11]:
for col in num_station_index.columns:
    if 'num_L0_mean' in col:
        print col

num_L0_mean_index_diff_0
num_L0_mean_index_diff_1


In [ ]:
single_important_features  =['L3_S30_F3754', 'L3_S30_F3759', 'L3_S33_F3857', 'L3_S33_F3859', 
                             'L3_S30_F3744', 'L3_S30_F3749', 'L3_S30_F3704', 'L3_S33_F3865']



#### build categorical features

In [10]:
def BasicCat_FeatureEngineering(train_cat):
    ## feature engineering on the date features
    encoder = preprocessing.LabelEncoder()
    column_names = train_cat.columns.tolist()
    column_names.append('NaN')
    encoder.fit(column_names)
    dat_new_fea = pd.DataFrame()
    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)
    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)
    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)
    dat_new_fea['cat_max'] = train_cat.max(axis=1)
    dat_new_fea['cat_min'] = train_cat.min(axis=1)
    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']
    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']

    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)
    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])
    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)
    dat_new_fea['cat_idxmin'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmin'] = encoder.transform(dat_new_fea['cat_idxmin'])
    return dat_new_fea



def encode_categorical_by_dep_var(train, test, dep_var_column='Response'):
    for col_name in train.columns:
        if col_name == dep_var_column:
            continue
        dep_var_mean = train[[col_name, dep_var_column]].groupby(col_name).mean()
    
        dep_var_dict = {}
        for level in dep_var_mean.index.tolist():
            dep_var_dict[level] = dep_var_mean.ix[level, dep_var_column]
    
        train[col_name] = train[col_name].replace(dep_var_dict)  
        test[col_name] = test[col_name].replace(dep_var_dict)  

In [11]:
start_time = time.time()

train_cat['Response'] = train_num['Response']
encode_categorical_by_dep_var(train_cat, test_cat)
train_cat.drop('Response', axis=1, inplace=True)

train_cat_Basics = BasicCat_FeatureEngineering(train_cat)
test_cat_Basics  = BasicCat_FeatureEngineering(train_cat)

print 'finish generating categorical features using {} seconds'.format(round(time.time() - start_time, 0))


finish generating categorical features using 16.0 seconds


In [12]:
print train_cat.shape, test_cat.shape
train_cat.head()

(50000, 1111) (50000, 1111)


,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4217,L3_S49_F4220,L3_S49_F4222,L3_S49_F4225,L3_S49_F4227,L3_S49_F4230,L3_S49_F4232,L3_S49_F4235,L3_S49_F4237,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
start_time = time.time()
combined_train_cat = pd.concat([train_cat, train_cat_Basics], axis=1)
combined_test_cat  = pd.concat([test_cat, test_cat_Basics], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} seconds'.format(round((time.time() - start_time), 2))

finish feature engineering date using 0.51 seconds


In [95]:
#### numerical feature engineering work
train_num_Basics = NumericalFeatureEngineering(train_num)
test_num_Basics = NumericalFeatureEngineering(test_num)

combined_train_num = pd.concat([train_num, train_num_Basics], axis=1)
combined_test_num  = pd.concat([test_num, test_num_Basics], axis=1)                                                                            
print 'combined train numerical feature shape: {}, combined test numerical features shape: {}'.format(combined_train_num.shape, combined_test_num.shape)

combined train numerical feature shape: (50000, 978), combined test numerical features shape: (50000, 977)


In [15]:
### section of date features

## basic features from tmp_train_dat
train_dat_Basics = BasicDate_FeatureEngineering(train_dat)
test_dat_Basics  = BasicDate_FeatureEngineering(test_dat)

In [16]:
## normalized date columns
train_dat_Norm = train_dat.apply(getRelativeTimeColumns, axis=1)
test_dat_Norm  = test_dat.apply(getRelativeTimeColumns, axis=1)
## remove single-valued columns
remove_single_value_columns(train_dat_Norm, test=test_dat_Norm)

raw train data dimension:  (50000, 1154)
raw test data dimension:  (50000, 1154)
processed train data dimension:  (50000, 972)
processed test data dimension:  (50000, 972)


In [17]:
encoder = preprocessing.LabelEncoder()
column_names = train_dat.columns.tolist()
column_names.append('NaN')
encoder.fit(column_names)

LabelEncoder()

In [18]:
## TimeDiff features
train_dat_TimeDiff = train_dat.apply(getTimeChangeColumns, axis=1)
test_dat_TimeDiff  = test_dat.apply(getTimeChangeColumns, axis=1)
TimeDiff_ColumnNames = ['time_diff_start_col', 'time_diff_end_col', 'time_diff_value',
                        'time_ratio_value', 'first_time_value', 'last_time_value', 'first_date_value']

train_dat_TimeDiff.columns = TimeDiff_ColumnNames
test_dat_TimeDiff.columns = TimeDiff_ColumnNames

for column in ['time_diff_start_col', 'time_diff_end_col']:
    train_dat_TimeDiff[column].fillna('NaN', inplace=True)
    train_dat_TimeDiff[column] = encoder.transform(train_dat_TimeDiff[column])
    
    test_dat_TimeDiff[column].fillna('NaN', inplace=True)
    test_dat_TimeDiff[column] = encoder.transform(test_dat_TimeDiff[column])


In [19]:
start_time = time.time()

## section to create timeStep features

unique_value_counts = 6
timeStep_columnNames = []
column_name_columns = []
for i in xrange(unique_value_counts):
    timeStep_columnNames.extend(['time_diff_step_{}'.format(i), 'column_counts_step_{}'.format(i),
                                 'time_cost_step_{}'.format(i), 'first_column_step_{}'.format(i)])
    column_name_columns.append('first_column_step_{}'.format(i))

train_dat_TimeStep = train_dat_Norm.apply(getTimeSteps, axis=1)
test_dat_TimeStep  = test_dat_Norm.apply(getTimeSteps, axis=1)
train_dat_TimeStep.columns = timeStep_columnNames
test_dat_TimeStep.columns  = timeStep_columnNames

for column in column_name_columns:
    train_dat_TimeStep[column].fillna('NaN', inplace=True)
    test_dat_TimeStep[column].fillna('NaN', inplace=True)
    train_dat_TimeStep[column] = encoder.transform(train_dat_TimeStep[column])
    test_dat_TimeStep[column] = encoder.transform(test_dat_TimeStep[column])


print 'finish generating TimeStep features using {} seconds'.format(round(time.time() - start_time, 0))


finish generating TimeStep features using 605.0 seconds


In [93]:
start_time = time.time()
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date using 0.01 minutes


In [94]:
print combined_train_dat.shape, combined_test_dat.shape
combined_train_dat.head()

(50000, 1013) (50000, 1013)


,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,L0_S0_D19,...,time_cost_step_3,first_column_step_3,time_diff_step_4,column_counts_step_4,time_cost_step_4,first_column_step_4,time_diff_step_5,column_counts_step_5,time_cost_step_5,first_column_step_5
Id,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.083971,945,5.72,29,0.197241,1018,NaN,0,0.0,1154
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1154,NaN,0,0.000000,1154,NaN,0,0.0,1154
401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1154,NaN,0,0.000000,1154,NaN,0,0.0,1154
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1154,NaN,0,0.000000,1154,NaN,0,0.0,1154
519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050385,945,3.94,19,0.207368,1028,NaN,0,0.0,1154


In [96]:
train_test_datIndex_features = build_IndexFeatures(combined_train_dat, combined_test_dat)

#### load the feature importances from xgboost models

In [50]:
xgb_feature_imp = pd.read_csv('/home/ymm/full_data_xgb_feature_importance.csv', index_col='feature')

In [51]:
xgb_feature_imp.head()

,fscore_0,norm_fscore_0,fscore_1,norm_fscore_1,fscore_2,norm_fscore_2,fscore_3,norm_fscore_3,fscore_4,norm_fscore_4,...,fscore_8,norm_fscore_8,fscore_9,norm_fscore_9,fscore_10,norm_fscore_10,fscore_11,norm_fscore_11,fscore_sum,norm_fscore_sum
feature,,,,,,,,,,,,,,,,,,,,,
first_time_value_index_diff_1,1357.0,0.013342,1480.0,0.014495,773.0,0.032962,1701.0,0.011999,1198.0,0.018810,...,1723.0,0.012308,747.0,0.031607,1671.0,0.011886,1152.0,0.018246,15033.0,0.228153
L3_S33_F3857,1317.0,0.012948,1226.0,0.012007,316.0,0.013475,1797.0,0.012676,648.0,0.010175,...,1745.0,0.012465,391.0,0.016544,1874.0,0.013330,708.0,0.011214,12276.0,0.149472
last_time_value_index_diff_1,1015.0,0.009979,1059.0,0.010372,410.0,0.017483,1310.0,0.009241,840.0,0.013189,...,1488.0,0.010629,382.0,0.016163,1329.0,0.009453,743.0,0.011768,10737.0,0.145426
L3_S33_F3859,1223.0,0.012024,1161.0,0.011371,264.0,0.011258,1722.0,0.012147,744.0,0.011682,...,1556.0,0.011115,375.0,0.015867,1649.0,0.011729,786.0,0.012449,11676.0,0.144343
last_time_value_index_diff_0,1020.0,0.010028,1087.0,0.010646,350.0,0.014925,1351.0,0.009530,707.0,0.011101,...,1391.0,0.009937,310.0,0.013117,1320.0,0.009389,612.0,0.009693,10258.0,0.132143


In [97]:
## sort by the norm_fscore_sum
sorted_combined_imp = xgb_feature_imp.sort_values(by=['norm_fscore_sum'], ascending=False)
#imp_feature = sorted_combined_imp.index[sorted_combined_imp['norm_fscore_sum'] >= 0.005].tolist()
imp_feature = sorted_combined_imp.index.tolist()

In [98]:
def select_important_features(df, imp_feature, test_df = None, dep_var_name = 'Response'):
    imp_col_names = [col for col in df.columns if col in imp_feature]
    print 'total {} columns in original DataFrame, select {} columns'.format(df.shape[1], len(imp_col_names))
    train_col_names = imp_col_names[:]
    test_col_names = imp_col_names[:]
    if dep_var_name in df.columns:    
        train_col_names.append(dep_var_name)
    if test_df is None:
        return df[train_col_names]
    else:
        return df[train_col_names], test_df[test_col_names]


In [99]:
combined_train_num, combined_test_num  = select_important_features(combined_train_num, imp_feature, combined_test_num)
combined_train_dat, combined_test_dat  = select_important_features(combined_train_dat, imp_feature, combined_test_dat)
train_test_datIndex_features = select_important_features(train_test_datIndex_features, imp_feature)

total 978 columns in original DataFrame, select 691 columns
total 1013 columns in original DataFrame, select 108 columns
total 23 columns in original DataFrame, select 22 columns


#### combine all the features together

In [48]:
## combined data with station features
#combined_train = pd.concat([train_dat_stations, train_num_stations, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([test_dat_stations, test_num_stations, combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [91]:
#combined_train = pd.concat([combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [100]:
## combined data with categorical features
#combined_train = pd.concat([combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [10]:
## combined data with all features
combined_train = pd.concat([combined_train_station_dat, combined_train_station_num, combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
combined_test  = pd.concat([combined_test_station_dat, combined_test_station_num, combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

NameError: name 'combined_train_station_dat' is not defined

In [105]:
print combined_test.shape
combined_test.head()

(100000, 2813)


,dat_L2_S28_mean,dat_L2_S28_max,dat_L2_S28_min,dat_L2_S28_var,dat_L3_S31_mean,dat_L3_S31_max,dat_L3_S31_min,dat_L3_S31_var,dat_L2_S26_mean,dat_L2_S26_max,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90086.000000,45481.0,-60087.0,-3232.0,-371803.0,-3232.0,-68013.0,-3232.0,-203442.0,-1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37387.000000,46999.5,-1365451.0,-62612.0,-2299392.0,-5943.0,-407847.0,-61348.0,1.0,-1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,704.11,704.11,...,21175.333333,666.0,-1962074.0,-24124.0,-1748713.0,-24124.0,-95311.0,-24124.0,1.0,-5.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.50,255.50,...,18799.800000,18017.2,-88702.0,-2088727.0,-74259.0,-2088727.0,-74259.0,-61577.0,-74259.0,-63560.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print combined_train.shape
combined_train.head()

NameError: name 'combined_train' is not defined

#### KFold cross-validation

In [63]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False


In [107]:

skf = StratifiedKFold(combined_train[dep_var_name], 4)

for train_index, valid_index in skf:
    valid_data = combined_train.iloc[valid_index]
    tmp_train  = combined_train.iloc[train_index]

    y = tmp_train[dep_var_name].values
    X = tmp_train.drop(dep_var_name, axis=1)

    valid_y = valid_data[dep_var_name].values
    valid_X = valid_data.drop(dep_var_name, axis=1)
    
    model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
    model.fit(tmp_train, dep_var_name)
    
    pred = model.predict(valid_X)
    print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
    print '\n'
    print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 170.228310502
a base_score 0.00584015573749 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.69995
[50]	train-auc:0.921853
[100]	train-auc:0.948515
[150]	train-auc:0.970543
[200]	train-auc:0.98424
[250]	train-auc:0.991027
[300]	train-auc:0.994345
[350]	train-auc:0.996518
[400]	train-auc:0.997822
[450]	train-auc:0.998533
[500]	train-auc:0.998926
the xgboost fit is finished by using 115.074739933 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00591952643788
threshold for preds: 0.0261300979575
0.170767009063


result from using flexsible threshold: (0.24373954271718642, 0.09077711403369904)
scale_pos_weight: 169.454545455
a base_score 0.00586666666667 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.667435
[50]	train-auc:0.927524
[100]	train-auc:0.946623
[150]	train-auc:0.970011
[200]	train-auc:0.985635
[250]	train-auc:0.991749
[300]	train-auc:0.995536
[350]	train-auc:0.997345
[400]	train-auc:0.99832
[450]	train-auc:0.998861
[500]	train-auc:0.999199
the xgboost fit is finished by using 70.169508934 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00584
threshold for preds: 0.0296218443333
0.159474895031


result from using flexsible threshold: (0.20817584490475483, 0.06661496311426163)
scale_pos_weight: 169.454545455
a base_score 0.00586666666667 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.651507
[50]	train-auc:0.902859
[100]	train-auc:0.927081
[150]	train-auc:0.955421
[200]	train-auc:0.974134
[250]	train-auc:0.984803
[300]	train-auc:0.989923
[350]	train-auc:0.994493
[400]	train-auc:0.996617
[450]	train-auc:0.997861
[500]	train-auc:0.99858
the xgboost fit is finished by using 69.8084080219 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00584
threshold for preds: 0.0210679047887
0.159474895031


result from using flexsible threshold: (0.1785420249134118, 0.029079917818307877)
scale_pos_weight: 169.459090909
a base_score 0.00586651022639 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.75029
[50]	train-auc:0.907086
[100]	train-auc:0.938593
[150]	train-auc:0.964457
[200]	train-auc:0.978273
[250]	train-auc:0.986418
[300]	train-auc:0.991929
[350]	train-auc:0.99486
[400]	train-auc:0.996692
[450]	train-auc:0.99765
[500]	train-auc:0.998312
the xgboost fit is finished by using 99.1968438625 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00584046723738
threshold for preds: 0.0221525590278
0.145695393442


result from using flexsible threshold: (0.20841519980584888, 0.048034001141786575)


In [8]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)

model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
model.fit(tmp_train, dep_var_name)
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

NameError: name 'combined_train' is not defined